## Import modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import csv
import sys, os

## Define util functions
* **get_data:** 
This is used to request currency data from local exchange. In this notebook pre-downloaded data stored in csv files are used to avoid any exposure of api keys

* **parse_calc_vol:** 
Main action happens here. It can process data source-dependent whether it is directly from exchange or pre-downladed csv files. Historical prices of given currency pair are processed. A dataframe including average daily and single mean volatility data is returned for given single currency. All steps in this process are explained as comments in code below.

In [ ]:
def get_data(currency_pair):
    return btcturk.ohlc(pairSymbol=currency_pair)

def parse_calc_vol(raw_ohlc=None, df_ohlc=None):
    # detect data source (direct from exchange or pre-downloaded)
    if raw_ohlc is not None:
        df = pd.DataFrame(raw_ohlc[:365])
    elif df_ohlc is not None:
        df = df_ohlc[:365]
    # get only "date" and "close" values
    df = df.loc[:, ['date', 'close']]
    # reverse dataframe 
    # (source: https://stackoverflow.com/questions/20444087/right-way-to-reverse-a-pandas-dataframe)
    df = df.iloc[::-1].reset_index(drop=True)
    # convert string close prices to int
    df["close"] = df["close"].astype(float)
    # remove rows with close price: 0 
    # (source: https://codereview.stackexchange.com/questions/185389/dropping-rows-from-a-pandas-dataframe-where-some-of-the-columns-have-value-0/185390)
    df = df[(df[['close']] != 0).all(axis=1)]
    # calculate pct changes aka daily returns
    df["return"] = df.close.pct_change()
    # scale returns 100x
    df['return_100x'] = np.multiply(df['return'], 100)
    # get volatility by absolute value of return pcts
    df["vol"] = df["return_100x"].abs()
    # remove NaN values
    df = df.dropna()
    return [df, df.vol.mean()]

## Process data for each currency

In [ ]:
curr_pair_vols = [] 

# read pre-downloaded historical currency data
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pair_full_path = os.path.join(dirname, filename)
        pair_name = filename.replace(".csv", "")
        # daily average volatility is calculated for single currency pair (e.g.: BTC_TRY)
        result = parse_calc_vol(df_ohlc = pd.read_csv(pair_full_path))
        # collect results for each pair in a list 
        curr_pair_vols.append([pair_name, result[1]])

# convert list of volatility results into dataframe
all_average_daily_volatils = pd.DataFrame(curr_pair_vols, columns = ['Currency_Name', 'Average_Daily_Volatility']) 

## Average volatilities of each currency based on their daily returns

In [ ]:
all_average_daily_volatils.sort_values(by="Average_Daily_Volatility",ascending=False)

#### Interestingly BTC seems stable relative to other currencies. Therefore it is plausible to focus on so called 'altcoins' for higher intraday return expectations 

## Details for single currency
### ADA_TRY

#### plot daily volatility values

In [ ]:
ada_vol_data = parse_calc_vol(pd.read_csv("/kaggle/input/historical-currency-data/DOT_TRY.csv"))
ada_vol_data[0].plot(x="date", y="vol")

In [ ]:
dot_vol_data[0].head()